# TP2 : Reconstruction tomographique
### Physique Numérique (PHY-3500)
### Par: Simon Gauthier, Laurent Émond, Thomas Charland
### Présenté à: Xavier Roy-Pomerleau
### Remis le: 23 mars 2025

## Mise en contexte

L'arrivée de l'ordinateur a révolutionné la médecine moderne en permettant de développer des méthodes nécessitant sur une puissance de calcul élevée. La tomodensitométrie (*Computer Tomography*, en anglais) est l'une de ces méthodes.

Ce document résout le TP2 du cours de Physique Numérique (PHY-3500). 

In [ ]:
#Importer toutes les libraries nécessaires pour la résolution du TP
import numpy as np
import matplotlib.pyplot as plt

# Numéro 1

In [ ]:
#Code pour résoudre le numéro 1

# Numéro 2

In [ ]:
#Code pour résoudre le numéro 2

# Numéro 3

In [ ]:
#Code pour résoudre le numéro 3

# Numéro 4

In [ ]:
#Code pour résoudre le numéro 4

# Numéro 5

In [ ]:
#Code pour résoudre le numéro 5

# Bibliographie

[1] TP2 - Reconstruction tomographique

[2]

[3]

[4]